### Archivo de creacion del mapa: FILTRO CAMION SEMANA

### 1_ Preparamos el entorno

**Importamos librerias**

In [1]:
# Preliminares
import re
import warnings

warnings.filterwarnings('ignore')

from math import radians

import folium
import matplotlib.cm as cm
import numpy as np
import openpyxl
import pandas as pd
from folium import CustomIcon
from folium.plugins import HeatMap, MarkerCluster
from matplotlib.colors import to_hex
from sklearn.metrics.pairwise import haversine_distances

**Cargamos los set**

In [2]:
# Dataframe Camiones estacionados
estacionados_camion = pd.read_excel("./estacionados_camion.xlsx", engine="openpyxl")

# Dataframe clientes 
ubi_cliente = pd.read_excel("./ubi_cliente.xlsx", engine="openpyxl")

# Tareas.xlsx
df_tareas = pd.read_excel('./Tareas-limpio.xlsx', engine="openpyxl")

**Cargamos ubicaciones extra Abitab,Red pagos, Empresa, Casas**

In [ ]:
# Abitab y red pagos
lugares_pago = {
        'CODIGO': [200000, 200002, 200003, 200005, 200006, 200007, 200008, 200009, 200010, 200011, 200012, 200013, 200014, 200015, 200016, 200017, 200018, 200019, 200020, 200021, 200022, 200023, 200024, 200025, 200026, 200027, 200028, 200029, 200030, 200031],
        'NOMCLI': ['Abitab', 'ABITAB 12/18', 'Abitab', 'Abitab Ruta 87', 'Abitab 12-21', 'Abitab', 'Abitab', 'Cambilex', 'Abitab', 'Abitab', 'Abitab', 'Abitab', 'Abitab', 'Abitab', 'Abitab Agencia 18/09', 'Redpagos Minimercado Nico', 'Red Pagos El Dorado, Las Toscas', 'Redpagos Bachino', 'Redpagos San Luis', 'Redpagos La Tuna', 'Redpagos Provision San Jorge', 'Red Pagos', 'RedPagos', 'RedPagos', 'Cambilex', 'Redpagos Salinas', 'Redpagos Salinas Norte', 'Redpagos Pinamar', 'Redpagos Agencias Migues', 'Redpagos Saucedo'],
        'LATITUD': [-34.7773749, -34.778762, -34.7769439, -34.7722784, -34.7717067, -34.7576624, -34.7714756, -34.772715, -34.7554596, -34.7673311, -34.7506218, -34.758673, -34.769283, -34.7701505, -34.6003052, -34.79286, -34.7710358, -34.7592078, -34.7663154, -34.7801148, -34.6832898, -34.7499177, -34.7741445, -34.7679195, -34.7773725, -34.7758541, -34.77187, -34.778569, -34.4884583, -34.5999389],
        'LONGITUD': [-55.8391627, -55.8620459, -55.8489382, -55.8413815, -55.8195923, -55.766793, -55.7620991, -55.7637416, -55.7246934, -55.7220688, -55.7094748, -55.677871, -55.6553415, -55.5763933, -55.4663981, -55.49211, -55.7337499, -55.6766295, -55.5899085, -55.5607396, -55.7024884, -55.7083951, -55.7615713, -55.767487, -55.8391636, -55.8397048, -55.84086, -55.862882, -55.630918, -55.4665898]
    }

# Crear DataFrame
lugares_pago_df = pd.DataFrame(lugares_pago)

# ___________________________________________________________________________

# Añadir lugares empresa **Codigo 200.000 a 200032**
ubi_empresa = {
        'CODIGO': [100001, 100002, 100003, 100004, 100005,100000,],
        'NOMCLI': ['CASA DARIO', 'CASA HUGO', 'CASA CONRADO', 'CASA MIGUEL', 'CASA MARTIN','EMPRESA'],
        'LATITUD': [-34.770962, -34.76416, -34.766187, -34.787831, -34.761527,-34.771148],
        'LONGITUD': [-55.742166, -55.86347, -55.780495, -55.850621, -55.74736, -55.758]
    }
df_ubi_empresa = pd.DataFrame(ubi_empresa)

# Coordenadas de "EMPRESA"
coords_empresa = [-34.771148, -55.758]

# Umbral de distancia para considerar que un camión está en "EMPRESA"
umbral_distancia = 0.00001


Iconos especiales camiones numero_placa

In [ ]:
# Diccionario de iconos personalizados (RUTA)
iconos_camion = {
    'AAW4251': '../num_placa_PNG/AAW4251.png',
    'BERLINGO7008': '../num_placa_PNG/BERLINGO7008.png',
    'BYD1006': '../num_placa_PNG/BYD1006.png',
    'CAA1076': '../num_placa_PNG/CAA1076.png'
}

# Normalizar las claves del diccionario de iconos
iconos_camion_normalizado = {key.replace(' ', '').upper(): value for key, value in iconos_camion.items()}

### 2_ Ajustes previos al mapeo

Cálculo de distancias (haversine)

In [3]:
def calcular_distancia(lat1, lon1, lat2, lon2):
    return haversine_distances([[radians(lat1), radians(lon1)], [radians(lat2), radians(lon2)]])[0][1]

In [4]:
def agregar_columna_semana(estacionados_camion):
    estacionados_camion['Semana'] = estacionados_camion['Tiempo_de_Inicio'].dt.strftime('%U')
    
    # Obtener semanas únicas y placas únicas
semanas = estacionados_camion['Semana'].unique()
unique_plates = estacionados_camion['Numero_de_placa'].unique()

# 4. Definir los límites del mapa
def filtrar_por_semana(estacionados_camion, semana):
    return estacionados_camion[estacionados_camion['Semana'] == semana]


# 5. Filtrado de datos por semana
def definir_limites_mapa(ubi_cliente_con_extras):
    min_lon, max_lon = ubi_cliente_con_extras['LONGITUD'].min(), ubi_cliente_con_extras['LONGITUD'].max()
    min_lat, max_lat = ubi_cliente_con_extras['LATITUD'].min(), ubi_cliente_con_extras['LATITUD'].max()
    return min_lat, max_lat, min_lon, max_lon

# 6. Calcular clientes visitados

def obtener_clientes_visitados(camiones_placa, ubi_cliente_con_extras):
    clientes_visitados = set()
    for _, camion in camiones_placa.iterrows():
        distancias = ubi_cliente_con_extras.apply(lambda row: calcular_distancia(
            camion['camion_x'], camion['camion_y'], row['LATITUD'], row['LONGITUD']), axis=1)
        cliente_cercano_idx = distancias.idxmin()
        cliente_cercano = ubi_cliente_con_extras.loc[cliente_cercano_idx]
        clientes_visitados.add(cliente_cercano['CODIGO'])
    return clientes_visitados


# Normalizar las patentes en el DataFrame
estacionados_camion['Numero_de_placa'] = estacionados_camion['Numero_de_placa'].str.replace(' ', '').str.upper()

In [6]:
# Ahora unimos los extras a ubi_cliente asegurando de que no se agreguen duplicados y que no se pierden los datos de ubi_cliente
ubi_cliente_con_extras = pd.concat([ubi_cliente, df_ubi_empresa, lugares_pago_df], ignore_index=True).drop_duplicates()

### 3_ MAPA


- Sin iconos de proyectos
- Sin "demora" 

In [14]:
# Definir los límites del mapa
min_lon, max_lon = ubi_cliente_con_extras['LONGITUD'].min(), ubi_cliente_con_extras['LONGITUD'].max()
min_lat, max_lat = ubi_cliente_con_extras['LATITUD'].min(), ubi_cliente_con_extras['LATITUD'].max()


# Convertir la columna 'Tiempo_de_Inicio' a tipo datetime si aún no lo es

# Añadir columna de semana al DataFrame
estacionados_camion['Semana'] = estacionados_camion['Tiempo_de_Inicio'].dt.strftime('%U')

# Crear un mapa para cada semana
semanas = estacionados_camion['Semana'].unique()

for semana in semanas:
    # Filtrar datos para la semana actual
    datos_semana = estacionados_camion[estacionados_camion['Semana'] == semana]
    
    
    # Crear el mapa base usando el promedio de las coordenadas y los límites
    mapa2 = folium.Map(
        location=[datos_semana['camion_x'].mean(), datos_semana['camion_y'].mean()],
        zoom_start=12,
        max_bounds=True,
        min_lat=min_lat,
        max_lat=max_lat,
        min_lon=min_lon,
        max_lon=max_lon
    )

    # Crear un diccionario de colores para cada número de placa de camión
    unique_plates = datos_semana['Numero_de_placa'].unique()
    colormap_camiones = cm.get_cmap('tab20', len(unique_plates))  # Puedes cambiar 'tab20' por otra paleta de colores
    color_dict = {plate: to_hex(colormap_camiones(i)) for i, plate in enumerate(unique_plates)}

    # Crear capas para cada número de placa
    for placa in unique_plates:
        capa_placa = folium.FeatureGroup(name=f"Placa {placa}", show=False)
        
        # Filtrar camiones por número de placa
        camiones_placa = datos_semana[datos_semana['Numero_de_placa'] == placa]
        
        clientes_visitados = set()
        
        # Convertir camiones de lista a DataFrame
        camiones_df = pd.DataFrame(camiones_placa)
        
        for _, camion in camiones_df.iterrows():
            patente = camion['Numero_de_placa']
            
            # Verificar si la patente existe en el diccionario normalizado
            if patente in iconos_camion_normalizado:
                icon_path = iconos_camion_normalizado[patente]
                icono = folium.CustomIcon(icon_image=icon_path, icon_size=(30, 30))
                
                folium.Marker(
                    location=[camion['camion_x'], camion['camion_y']],
                    popup=f"<b>Camión: {patente}</b><br>Tiempo de Inicio: {camion['Tiempo_de_Inicio']}<br>Duración: {camion['Duracion']}<br>Tiempo Final: {camion['Tiempo_Final']}",
                    tooltip=f"Camión: {patente}",
                    icon=icono
                ).add_to(capa_placa)
            else:
                print(f"Patente no encontrada en el diccionario: {patente}")
            
            # Encontrar el cliente más cercano
            distancias = ubi_cliente.apply(lambda row: calcular_distancia(
                camion['camion_x'], camion['camion_y'], row['LATITUD'], row['LONGITUD']), axis=1)
            cliente_cercano_idx = distancias.idxmin()
            cliente_cercano = ubi_cliente.loc[cliente_cercano_idx]
            clientes_visitados.add(cliente_cercano['CODIGO'])
            
            # Dibujar una línea entre el camión y el cliente más cercano
            folium.PolyLine(
                locations=[[camion['camion_x'], camion['camion_y']], 
                        [cliente_cercano['LATITUD'], cliente_cercano['LONGITUD']]],
                color="blue",
                weight=2,
                opacity=0.8
            ).add_to(capa_placa)
        
        # Añadir marcadores para los clientes visitados
        for _, cliente in ubi_cliente.iterrows():
            if cliente['CODIGO'] in clientes_visitados:
                folium.Marker(
                    location=[cliente['LATITUD'], cliente['LONGITUD']],
                    popup=f"<b>Cliente: {cliente['CODIGO']}</b><br>Nombre: {cliente['NOMCLI']}",
                    tooltip=f"CODIGO: {cliente['CODIGO']}<br>Nombre: {cliente['NOMCLI']}",
                    icon=folium.Icon(color='red', icon='home', prefix='fa')
                ).add_to(capa_placa)
        
        capa_placa.add_to(mapa2)



    # Añadir ubicaciones de la empresa
    for _, empresa in df_ubi_empresa.iterrows():
        folium.Circle(
            location=[empresa['LATITUD'], empresa['LONGITUD']],
            radius=100,
            color="blue",
            fill=True,
            fill_opacity=0.3
        ).add_to(mapa2)
        
        
        folium.Marker(
            location=[empresa['LATITUD'], empresa['LONGITUD']],
            popup=f"<b>Empresa: {empresa['NOMCLI']}</b>",
            tooltip=f"CODIGO: {empresa['CODIGO']}<br>Nombre: {empresa['NOMCLI']}",
            icon=folium.Icon(color='blue', icon='fa-truck', prefix='fa')
        ).add_to(mapa2)

    # Añadir nuevos lugares de pago
    for _, lugar_pago in lugares_pago_df.iterrows():
        folium.Circle(
            location=[lugar_pago['LATITUD'], lugar_pago['LONGITUD']],
            radius=100,
            color="green",
            fill=True,
            fill_opacity=0.3
        ).add_to(mapa2)
        
        folium.Marker(
            location=[lugar_pago['LATITUD'], lugar_pago['LONGITUD']],
            popup=f"<b>Cliente: {lugar_pago['NOMCLI']}</b>",
            tooltip=lugar_pago['NOMCLI'],
            icon=folium.Icon(color="green", icon="dollar-sign",prefix='fa')
        ).add_to(mapa2)
        
        lugar_pago


    # Añadir control de capas
    folium.LayerControl().add_to(mapa2)

    # Guardar el mapa en un archivo HTML
    mapa_semana_path = f"../mapas_html/mapa_semana_{int(semana)}_placa.html"
    mapa2.save(mapa_semana_path)
    print(f"Mapa de la semana {semana} guardado en {mapa_semana_path}")

Mapa de la semana 30 guardado en ../mapas_html/mapa_semana_30_placa.html
Mapa de la semana 31 guardado en ../mapas_html/mapa_semana_31_placa.html


KeyboardInterrupt: 

### Revisar si sirve reciclar algo de este codigo

In [ ]:


# Convertir la columna 'Tiempo_de_Inicio' a tipo datetime si aún no lo es

# Añadir columna de semana al DataFrame
estacionados_camion['Semana'] = estacionados_camion['Tiempo_de_Inicio'].dt.strftime('%U')

# Crear un mapa para cada semana
semanas = estacionados_camion['Semana'].unique()

for semana in semanas:
    # Filtrar datos para la semana actual
    datos_semana = estacionados_camion[estacionados_camion['Semana'] == semana]
    
    
    # Crear el mapa base usando el promedio de las coordenadas y los límites
    mapa2 = folium.Map(
        location=[datos_semana['camion_x'].mean(), datos_semana['camion_y'].mean()],
        zoom_start=12,
        max_bounds=True,
        min_lat=min_lat,
        max_lat=max_lat,
        min_lon=min_lon,
        max_lon=max_lon
    )

    # Crear un diccionario de colores para cada número de placa de camión
    unique_plates = datos_semana['Numero_de_placa'].unique()
    colormap_camiones = cm.get_cmap('tab20', len(unique_plates))  # Puedes cambiar 'tab20' por otra paleta de colores
    color_dict = {plate: to_hex(colormap_camiones(i)) for i, plate in enumerate(unique_plates)}

    # Crear capas para cada número de placa
    for placa in unique_plates:
        capa_placa = folium.FeatureGroup(name=f"Placa {placa}", show=False)
        
        # Filtrar camiones por número de placa
        camiones_placa = datos_semana[datos_semana['Numero_de_placa'] == placa]
        
        clientes_visitados = set()
        
        # Convertir camiones de lista a DataFrame
        camiones_df = pd.DataFrame(camiones_placa)
        
        for _, camion in camiones_df.iterrows():
            patente = camion['Numero_de_placa']
            
            # Verificar si la patente existe en el diccionario normalizado
            if patente in iconos_camion_normalizado:
                icon_path = iconos_camion_normalizado[patente]
                icono = folium.CustomIcon(icon_image=icon_path, icon_size=(30, 30))
                
                folium.Marker(
                    location=[camion['camion_x'], camion['camion_y']],
                    popup=f"<b>Camión: {patente}</b><br>Tiempo de Inicio: {camion['Tiempo_de_Inicio']}<br>Duración: {camion['Duracion']}<br>Tiempo Final: {camion['Tiempo_Final']}",
                    tooltip=f"Camión: {patente}",
                    icon=icono
                ).add_to(capa_placa)
            else:
                print(f"Patente no encontrada en el diccionario: {patente}")
            
            # Encontrar el cliente más cercano
            distancias = ubi_cliente.apply(lambda row: calcular_distancia(
                camion['camion_x'], camion['camion_y'], row['LATITUD'], row['LONGITUD']), axis=1)
            cliente_cercano_idx = distancias.idxmin()
            cliente_cercano = ubi_cliente.loc[cliente_cercano_idx]
            clientes_visitados.add(cliente_cercano['CODIGO'])
            
            # Dibujar una línea entre el camión y el cliente más cercano
            folium.PolyLine(
                locations=[[camion['camion_x'], camion['camion_y']], 
                        [cliente_cercano['LATITUD'], cliente_cercano['LONGITUD']]],
                color="blue",
                weight=2,
                opacity=0.8
            ).add_to(capa_placa)
        
        # Añadir marcadores para los clientes visitados
        for _, cliente in ubi_cliente.iterrows():
            if cliente['CODIGO'] in clientes_visitados:
                folium.Marker(
                    location=[cliente['LATITUD'], cliente['LONGITUD']],
                    popup=f"<b>Cliente: {cliente['CODIGO']}</b><br>Nombre: {cliente['NOMCLI']}",
                    tooltip=f"CODIGO: {cliente['CODIGO']}<br>Nombre: {cliente['NOMCLI']}",
                    icon=folium.Icon(color='red', icon='home', prefix='fa')
                ).add_to(capa_placa)
        
        capa_placa.add_to(mapa2)



    # Añadir ubicaciones de la empresa
    for _, empresa in df_ubi_empresa.iterrows():
        folium.Circle(
            location=[empresa['LATITUD'], empresa['LONGITUD']],
            radius=100,
            color="blue",
            fill=True,
            fill_opacity=0.3
        ).add_to(mapa2)
        
        
        folium.Marker(
            location=[empresa['LATITUD'], empresa['LONGITUD']],
            popup=f"<b>Empresa: {empresa['NOMCLI']}</b>",
            tooltip=f"CODIGO: {empresa['CODIGO']}<br>Nombre: {empresa['NOMCLI']}",
            icon=folium.Icon(color='blue', icon='fa-truck', prefix='fa')
        ).add_to(mapa2)

    # Añadir nuevos lugares de pago
    for _, lugar_pago in lugares_pago_df.iterrows():
        folium.Circle(
            location=[lugar_pago['LATITUD'], lugar_pago['LONGITUD']],
            radius=100,
            color="green",
            fill=True,
            fill_opacity=0.3
        ).add_to(mapa2)
        
        folium.Marker(
            location=[lugar_pago['LATITUD'], lugar_pago['LONGITUD']],
            popup=f"<b>Cliente: {lugar_pago['NOMCLI']}</b>",
            tooltip=lugar_pago['NOMCLI'],
            icon=folium.Icon(color="green", icon="dollar-sign",prefix='fa')
        ).add_to(mapa2)
        
        lugar_pago


    # Añadir control de capas
    folium.LayerControl().add_to(mapa2)

    # Guardar el mapa en un archivo HTML
    mapa_semana_path = f"../mapas_html/Julio/mapa_semana_{int(semana)}_placa.html"
    mapa2.save(mapa_semana_path)
    print(f"Mapa de la semana {semana} guardado en {mapa_semana_path}")

Ese mapa funciona pero: Muestra los lugares de pago como areas rojas, y no muestras los dato de la empresa, las ubicaciones no existen.

In [ ]:
# Filtrar por semanas del mes
semanas_del_mes = estacionados_camion['Semana_del_mes'].unique()

# Crear un mapa por cada semana
for semana in semanas_del_mes:
    # Filtrar datos para la semana actual
    datos_semana = estacionados_camion[estacionados_camion['Semana_del_mes'] == semana]

    # Definir los límites del mapa
    min_lon, max_lon = ubi_cliente['LONGITUD'].astype(float).min(), ubi_cliente['LONGITUD'].astype(float).max()
    min_lat, max_lat = ubi_cliente['LATITUD'].min(), ubi_cliente['LATITUD'].max()

    # Crear el mapa base usando el promedio de las coordenadas y los límites
    mapa = folium.Map(
        location=[datos_semana['camion_x'].mean(), datos_semana['camion_y'].mean()],
        zoom_start=12,
        max_bounds=True,
        min_lat=min_lat,
        max_lat=max_lat,
        min_lon=min_lon,
        max_lon=max_lon
    )

    # Crear un diccionario de colores para cada número de placa de camión
    unique_plates = datos_semana['Numero_de_placa'].unique()
    colormap_camiones = cm.get_cmap('tab20', len(unique_plates))  # Puedes cambiar 'tab20' por otra paleta de colores
    color_dict = {plate: to_hex(colormap_camiones(i)) for i, plate in enumerate(unique_plates)}

    # Crear un diccionario para almacenar los datos de los camiones por fecha
    camiones_por_fecha = {}
    for idx, camion in datos_semana.iterrows():
        fecha = camion['Tiempo_de_Inicio'].date()
        if fecha not in camiones_por_fecha:
            camiones_por_fecha[fecha] = []
        camiones_por_fecha[fecha].append(camion)

    # Crear capas para cada fecha con colores específicos para cada camión
    for fecha, camiones in camiones_por_fecha.items():
        # Convertir la fecha a un formato más amigable
        dia_del_mes = fecha.day
        capa_nombre = f"Día {dia_del_mes:02d}"  # Formato "Día 01", "Día 02", etc.
        
        capa_fecha = folium.FeatureGroup(name=capa_nombre, show=False)
        
        clientes_visitados = set()
        
        # Convertir camiones de lista a DataFrame
        camiones_df = pd.DataFrame(camiones)
        
        for _, camion in camiones_df.iterrows():
            patente = camion['Numero_de_placa']
            
            # Verificar si la patente existe en el diccionario normalizado
            if patente in iconos_camion_normalizado:
                icon_path = iconos_camion_normalizado[patente]
                icono = folium.CustomIcon(icon_image=icon_path, icon_size=(30, 30))
                
                folium.Marker(
                    location=[camion['camion_x'], camion['camion_y']],
                    popup=f"<b>Camión: {patente}</b><br>Tiempo de Inicio: {camion['Tiempo_de_Inicio']}<br>Duración: {camion['Duracion']}<br>Tiempo Final: {camion['Tiempo_Final']}",
                    tooltip=f"Camión: {patente}",
                    icon=icono
                ).add_to(capa_fecha)
            else:
                print(f"Patente no encontrada en el diccionario: {patente}")
            
            # Encontrar el cliente más cercano
            distancias = ubi_cliente.apply(lambda row: calcular_distancia(
                camion['camion_x'], camion['camion_y'], row['LATITUD'], row['LONGITUD']), axis=1)
            cliente_cercano_idx = distancias.idxmin()
            cliente_cercano = ubi_cliente.loc[cliente_cercano_idx]
            clientes_visitados.add(cliente_cercano['CODIGO'])
            
            # Dibujar una línea entre el camión y el cliente más cercano
            folium.PolyLine(
                locations=[[camion['camion_x'], camion['camion_y']], 
                           [cliente_cercano['LATITUD'], cliente_cercano['LONGITUD']]],
                color="blue",
                weight=2,
                opacity=0.8
            ).add_to(capa_fecha)
        
        # Añadir marcadores para los clientes visitados en la fecha específica
        for _, cliente in ubi_cliente.iterrows():
            if cliente['CODIGO'] in clientes_visitados:
                folium.Marker(
                    location=[cliente['LATITUD'], cliente['LONGITUD']],
                    popup=f"<b>Cliente: {cliente['CODIGO']}</b><br>Nombre: {cliente['NOMCLI']}<br>Fecha: {fecha}",
                    tooltip=f"CODIGO: {cliente['CODIGO']}<br>Nombre: {cliente['NOMCLI']}",
                    icon=folium.Icon(color='red', icon='home', prefix='fa')
                ).add_to(capa_fecha)
        
        capa_fecha.add_to(mapa)


    # Añadir ubicaciones de la empresa
    for _, empresa in df_ubi_empresa.iterrows():
        folium.Circle(
            location=[empresa['LATITUD'], empresa['LONGITUD']],
            radius=100,
            color="blue",
            fill=True,
            fill_opacity=0.3
        ).add_to(mapa)
        
        
        folium.Marker(
            location=[empresa['LATITUD'], empresa['LONGITUD']],
            popup=f"<b>Empresa: {empresa['NOMCLI']}</b>",
            tooltip=f"CODIGO: {empresa['CODIGO']}<br>Nombre: {empresa['NOMCLI']}",
            icon=folium.Icon(color='blue', icon='fa-truck', prefix='fa')
        ).add_to(mapa)

    # Añadir nuevos lugares de pago
    for _, lugar_pago in lugares_pago_df.iterrows():
        folium.Circle(
            location=[lugar_pago['LATITUD'], lugar_pago['LONGITUD']],
            radius=100,
            color="green",
            fill=True,
            fill_opacity=0.3
        ).add_to(mapa)
        
        folium.Marker(
            location=[lugar_pago['LATITUD'], lugar_pago['LONGITUD']],
            popup=f"<b>Cliente: {lugar_pago['NOMCLI']}</b>",
            tooltip=lugar_pago['NOMCLI'],
            icon=folium.Icon(color="green", icon="dollar-sign",prefix='fa')
        ).add_to(mapa)
        
        lugar_pago


    # Añadir control de capas
    folium.LayerControl().add_to(mapa)

    # Guardar el mapa
    mapa.save(f"../mapas_html/Agosto/mapa_semana_{semana}_08.html")


Ese mapa funciona pero: Muestra los lugares de pago como areas rojas, y no muestras los dato de la empresa, las ubicaciones no existen.